In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv("bank.csv")

In [3]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,0
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,0
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,0
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,0
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
age          4521 non-null int64
job          4521 non-null object
marital      4521 non-null object
education    4521 non-null object
default      4521 non-null object
balance      4521 non-null int64
housing      4521 non-null object
loan         4521 non-null object
contact      4521 non-null object
day          4521 non-null int64
month        4521 non-null object
duration     4521 non-null int64
campaign     4521 non-null int64
pdays        4521 non-null int64
previous     4521 non-null int64
poutcome     4521 non-null object
y            4521 non-null int64
dtypes: int64(8), object(9)
memory usage: 600.5+ KB


In [6]:
len(df.columns[df.dtypes == "object"])

9

Видим, что в задаче много категориальных признаков

In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
enc=LabelEncoder()

In [9]:
df_new=df.copy()

In [11]:
for col in df.columns[df.dtypes == "object"]:
    df_new[col] = enc.fit_transform(df[col])

In [12]:
df_new.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,10,1,0,0,1787,0,0,0,19,10,79,1,-1,0,3,0
1,33,7,1,1,0,4789,1,1,0,11,8,220,1,339,4,0,0
2,35,4,2,2,0,1350,1,0,0,16,0,185,1,330,1,0,0
3,30,4,1,2,0,1476,1,1,2,3,6,199,4,-1,0,3,0
4,59,1,1,1,0,0,1,0,2,5,8,226,1,-1,0,3,0


In [15]:
from sklearn.model_selection import train_test_split

In [21]:
x_train, x_test, y_train, y_test = train_test_split(df_new.iloc[:,:-1], df_new.iloc[:,-1], test_size = 0.3, random_state=42)

In [25]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [23]:
xgb_clf=XGBClassifier()

In [24]:
xgb_clf.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [26]:
accuracy_score(y_test, xgb_clf.predict(x_test))

0.8968312453942521

In [27]:
print(classification_report(y_test, xgb_clf.predict(x_test)))

              precision    recall  f1-score   support

           0       0.92      0.97      0.94      1205
           1       0.57      0.30      0.40       152

   micro avg       0.90      0.90      0.90      1357
   macro avg       0.75      0.64      0.67      1357
weighted avg       0.88      0.90      0.88      1357



# One-hot Encoding (Бинаризация признаков)

In [29]:
df_dummies=pd.get_dummies(df, columns=df.columns[df.dtypes == "object"])

In [30]:
df_dummies.columns

Index(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous',
       'y', 'job_admin.', 'job_blue-collar', 'job_entrepreneur',
       'job_housemaid', 'job_management', 'job_retired', 'job_self-employed',
       'job_services', 'job_student', 'job_technician', 'job_unemployed',
       'job_unknown', 'marital_divorced', 'marital_married', 'marital_single',
       'education_primary', 'education_secondary', 'education_tertiary',
       'education_unknown', 'default_no', 'default_yes', 'housing_no',
       'housing_yes', 'loan_no', 'loan_yes', 'contact_cellular',
       'contact_telephone', 'contact_unknown', 'month_apr', 'month_aug',
       'month_dec', 'month_feb', 'month_jan', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'poutcome_failure', 'poutcome_other', 'poutcome_success',
       'poutcome_unknown'],
      dtype='object')

In [31]:
df_dummies.shape

(4521, 52)

In [33]:
x_train, x_test, y_train, y_test = train_test_split(df_dummies.drop(columns="y", axis=1), df_dummies["y"], test_size = 0.3, random_state=42)

In [34]:
xgb_clf.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [35]:
accuracy_score(y_test, xgb_clf.predict(x_test))

0.9005158437730287

# Catboost

In [36]:
from catboost import CatBoostClassifier

In [40]:
cat_feat_ind=df.columns[df.dtypes == "object"].tolist()

In [41]:
cat_feat_ind

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'poutcome']

In [43]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns="y", axis=1), df["y"], test_size = 0.3, random_state=42)

In [42]:
cat_cls=CatBoostClassifier()

In [45]:
cat_cls.fit(x_train, y_train, cat_features=cat_feat_ind, silent=True)

In [46]:
accuracy_score(y_test, cat_cls.predict(x_test))

0.8953574060427414